# Spark

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark
 
spark = (SparkSession
         .builder
         .master("local")
         .appName("arrival")
         # Add MYSQL jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/mysql/usr/share/java/mysql-connector-java-8.0.30.jar")
         .getOrCreate())
sc = spark.sparkContext

22/11/29 08:40:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/29 08:40:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
query = "SELECT table_name FROM information_schema.tables WHERE table_type = 'base table'"

table_names = (spark.read.format("jdbc")
      .option("url","jdbc:mysql://202.129.206.52:3306/tsb_gps_db")
      .option("driver","com.mysql.cj.jdbc.Driver")
      .option("user","tsb_user")
      .option("password","tSb@2022!")
      .option("query", query)
      .load())
table_names.show(), table_names.count()

+--------------+
|    table_name|
+--------------+
|data_report_13|
|data_report_14|
|data_report_16|
|data_report_19|
|data_report_22|
|data_report_35|
|data_report_41|
|data_report_42|
|data_report_43|
|data_report_45|
|data_report_47|
|data_report_50|
+--------------+



(None, 12)

In [3]:
len(table_names.toPandas())

12

In [4]:
# END DATE
from tqdm import tqdm
table = []
time_all = []
for i in tqdm(range(len(table_names.toPandas()))):
    query = f"""
    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.toPandas().iloc[i][0]}
    order by utc DESC
    limit 5
    """

    df = (spark.read.format("jdbc")
          .option("url","jdbc:mysql://202.129.206.52:3306/tsb_gps_db")
          .option("driver","com.mysql.cj.jdbc.Driver")
          .option("user","tsb_user")
          .option("password","tSb@2022!")
          .option("query", query)
          .load())
    df = df.toPandas()
    if len(df) > 0:
        if table_names.toPandas().iloc[i][0] == 'data_report_19':
            time = df.iloc[1].utc
    #         print(table_names.toPandas().iloc[i][0], time)
            table.append(table_names.toPandas().iloc[i][0])
            time_all.append(time)
        else:
            time = df.iloc[0].utc
    #         print(table_names.toPandas().iloc[i][0], time)
            table.append(table_names.toPandas().iloc[i][0])
            time_all.append(time)

100%|██████████| 12/12 [00:39<00:00,  3.29s/it]                                 


In [5]:
import pandas as pd
end = pd.DataFrame([table, time_all]).T
end.columns = ['data_report', 'end_time']
end

,data_report,end_time
0,data_report_13,2022-09-21 20:59:52
1,data_report_14,2022-09-21 20:52:20
2,data_report_16,2022-09-21 20:55:24
3,data_report_19,2022-09-21 21:09:14
4,data_report_22,2022-09-21 21:09:39
5,data_report_35,2022-09-21 21:19:43
6,data_report_41,2022-09-21 21:19:42
7,data_report_42,2022-09-21 21:18:56
8,data_report_45,2022-09-21 21:03:13
9,data_report_47,2022-09-21 21:49:31


In [6]:
# END DATE
from tqdm import tqdm
table = []
time_all = []
for i in tqdm(range(len(table_names.toPandas()))):
    query = f"""
    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.toPandas().iloc[i][0]}
    order by utc ASC
    limit 5
    """

    df = (spark.read.format("jdbc")
          .option("url","jdbc:mysql://202.129.206.52:3306/tsb_gps_db")
          .option("driver","com.mysql.cj.jdbc.Driver")
          .option("user","tsb_user")
          .option("password","tSb@2022!")
          .option("query", query)
          .load())
    df = df.toPandas()
    if len(df) > 0:
        if table_names.toPandas().iloc[i][0] == 'data_report_19':
            time = df.iloc[1].utc
    #         print(table_names.toPandas().iloc[i][0], time)
            table.append(table_names.toPandas().iloc[i][0])
            time_all.append(time)
        else:
            time = df.iloc[0].utc
    #         print(table_names.toPandas().iloc[i][0], time)
            table.append(table_names.toPandas().iloc[i][0])
            time_all.append(time)

100%|██████████| 12/12 [00:36<00:00,  3.08s/it]                                 


In [7]:
import pandas as pd
start = pd.DataFrame([table, time_all]).T
start.columns = ['data_report', 'start_time']
start

,data_report,start_time
0,data_report_13,2022-03-25 00:00:23
1,data_report_14,2022-03-25 00:00:59
2,data_report_16,2022-05-07 15:52:45
3,data_report_19,2022-06-22 14:39:44
4,data_report_22,2022-03-24 00:00:01
5,data_report_35,2022-03-25 00:00:05
6,data_report_41,2022-03-25 00:00:57
7,data_report_42,2022-03-25 00:01:02
8,data_report_45,2022-03-24 00:11:06
9,data_report_47,2022-03-21 07:12:22


In [8]:
start_end_time = start.merge(end, left_on='data_report', right_on='data_report')
start_end_time

,data_report,start_time,end_time
0,data_report_13,2022-03-25 00:00:23,2022-09-21 20:59:52
1,data_report_14,2022-03-25 00:00:59,2022-09-21 20:52:20
2,data_report_16,2022-05-07 15:52:45,2022-09-21 20:55:24
3,data_report_19,2022-06-22 14:39:44,2022-09-21 21:09:14
4,data_report_22,2022-03-24 00:00:01,2022-09-21 21:09:39
5,data_report_35,2022-03-25 00:00:05,2022-09-21 21:19:43
6,data_report_41,2022-03-25 00:00:57,2022-09-21 21:19:42
7,data_report_42,2022-03-25 00:01:02,2022-09-21 21:18:56
8,data_report_45,2022-03-24 00:11:06,2022-09-21 21:03:13
9,data_report_47,2022-03-21 07:12:22,2022-09-21 21:49:31


# mysql.connector

## Vehicles

In [25]:
import pandas as pd
vehicles = pd.read_csv('/home/jovyan/work/data/vehicles/original_vehicles.csv')
vehicles.head()

,num,routes,chasis,gps
0,16-5840 กทม.,35,MRSBCREM3MZM00001,868998031822683
1,16-5843 กทม.,35,MRSBCREM5MZM00033,864507039673982
2,16-5845 กทม.,35,MRSBCREM8MZM00057,868998030634774
3,16-5844 กทม.,35,MRSBCREM1MZM00062,868998030615351
4,16-5847 กทม.,35,MRSBCREM7MZM00034,868998030627521


In [38]:
smart_bus = pd.read_excel('/home/jovyan/work/data/smart_bus.xlsx')
smart_bus['routes'] = smart_bus['สายรถ'].str.split('-', n=1).str.get(0)
smart_bus = smart_bus.drop('No.',axis=1)
smart_bus.head()

,ชื่อผู้ประกอบการ,ยี่ห้อรถ,ประเภทรถ,สายรถ,ทะเบียนรถ,จังหวัด,หมายเลขตัวถัง,วันที่ติดตั้ง,อีมี่ GPS,routes
0,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-01,16-4754,กรุงเทพมหานคร,PRAT29A3UFR000306,2565-07-04 00:00:00,866728061461934,28
1,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-02,16-4768,กรุงเทพมหานคร,PRAT29A3UFR000308,2565-07-04 00:00:00,866728061462346,28
2,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-03,16-4731,กรุงเทพมหานคร,PRAT29A3UFR000309,2565-07-04 00:00:00,867648045405519,28
3,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-04,16-4728,กรุงเทพมหานคร,PRAT29A3UFR000311,2565-07-04 00:00:00,866728061463203,28
4,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-05,16-4756,กรุงเทพมหานคร,PRAT29A3UFR000313,2565-07-04 00:00:00,866728061462437,28


In [39]:
v_list = vehicles.gps.tolist()
sb_list = smart_bus['อีมี่ GPS'].tolist()
vehicle_gps_list = v_list+sb_list

v_num_list = vehicles.num.tolist()
sb_num_list = smart_bus['ทะเบียนรถ'].tolist()
vehicle_num_list = v_num_list + sb_num_list

v_route_list = vehicles.routes.tolist()
sb_route_list = smart_bus['routes'].tolist()
vehicle_route_list = v_route_list + sb_route_list

v_chasis_list = vehicles.chasis.tolist()
sb_chasis_list = smart_bus['หมายเลขตัวถัง'].tolist()
vehicle_chasis_list = v_chasis_list + sb_chasis_list

all_v = pd.DataFrame(vehicle_gps_list, columns = ['gps_imei'])
all_v['num'] = vehicle_num_list
all_v['routes'] = vehicle_route_list
all_v['chasis'] = vehicle_chasis_list
all_v

,gps_imei,num,routes,chasis
0,868998031822683,16-5840 กทม.,35,MRSBCREM3MZM00001
1,864507039673982,16-5843 กทม.,35,MRSBCREM5MZM00033
2,868998030634774,16-5845 กทม.,35,MRSBCREM8MZM00057
3,868998030615351,16-5844 กทม.,35,MRSBCREM1MZM00062
4,868998030627521,16-5847 กทม.,35,MRSBCREM7MZM00034
...,...,...,...,...
452,867648045388376,16-3920,69,PRAT29A3UFR000476
453,867648045375092,16-3904,69,PRAT29A3UFR000477
454,867648045376983,16-3887,69,PRAT29A3UFR000478
455,867648045407127,16-3695,69,PRAT29A3UFR000439


In [40]:
all_v.routes.unique()

array([35, 39, 80, 133, 56, 6, '28', ' 29', '104', '113', '115', '124',
       '147L', '147R', '150', '167', '29', '40', '48', '504', '507', '51',
       '52', '538', '547', '554', '558', '69', '690'], dtype=object)

In [41]:
# all_v.to_csv('/home/jovyan/work/data/all_vehicle.csv', index=False)

In [42]:
all_v = pd.read_csv('/home/jovyan/work/data/all_vehicle.csv')
all_v.routes.unique()

array(['35', '39', '80', '133', '56', '6', '28', ' 29', '104', '113',
       '115', '124', '147L', '147R', '150', '167', '29', '40', '48',
       '504', '507', '51', '52', '538', '547', '554', '558', '69', '690'],
      dtype=object)

## Vehicles Abpo

In [32]:
import pandas as pd
all_v = pd.read_csv('/home/jovyan/work/data/vehicles/vehicles_30-11-22.csv')
all_v

,gps_imei,registerNumber,chassisNumber,motorNumber,brand,status,description,route
0,868998030645952,16-5868 กทม.,MRSBCREM3MZM00046,MCS20111197,MINE BUS,active,เคหะบางพลี – สถานีรถไฟฟ้า (BTS) เอกมัย,3-15(133)
1,864507039641906,16-5866 กทม.,MRSBCREM5MZM00050,MCS20111203,MINE BUS,active,เคหะบางพลี – สถานีรถไฟฟ้า (BTS) เอกมัย,3-15(133)
2,868998030588186,16-5871 กทม.,MRSBCREM5MZM00078,MCS20111122,MINE BUS,active,เคหะบางพลี – สถานีรถไฟฟ้า (BTS) เอกมัย,3-15(133)
3,866381052370324,16-5948 กทม.,MRSBCREM0MZM00067,MCS20111103,MINE BUS,active,เคหะบางพลี – สถานีรถไฟฟ้า (BTS) เอกมัย,3-15(133)
4,866381052370019,16-5905 กทม.,MRSBCREM8MZM00009,MCS20111160,MINE BUS,active,เคหะบางพลี – สถานีรถไฟฟ้า (BTS) เอกมัย,3-15(133)
...,...,...,...,...,...,...,...,...
892,864606049307746,16-6664,MRSBCREM2NZM00475,DPPC780010,MINE,active,ท่าอิฐ – มหาวิทยาลัยรามคำแหง (ทางด่วน),2-18E
893,864606049327587,16-6659,MRSBCREM8NZM00478,DPPC780013,MINE,active,ท่าอิฐ – มหาวิทยาลัยรามคำแหง (ทางด่วน),2-18E
894,864606049306862,16-6667,MRSBCREM6NZM00480,DPPC780015,MINE,active,ท่าอิฐ – มหาวิทยาลัยรามคำแหง (ทางด่วน),2-18E
895,864606049310534,16-6645,MRSBCREM8NZM00481,DPPC780016,MINE,active,ท่าอิฐ – มหาวิทยาลัยรามคำแหง (ทางด่วน),2-18E


In [14]:
import mysql.connector
import pandas as pd
conn = mysql.connector.connect(user='tsb_user', database='tsb_gps_db',
                               password='tSb@2022!',
                               host="202.129.206.52",
                               port=3306)
# cursor = conn.cursor()
query = "SELECT table_name FROM information_schema.tables WHERE table_type = 'base table'"
table_names = pd.read_sql(query, con=conn)
table_names

,table_name
0,data_report_13
1,data_report_14
2,data_report_16
3,data_report_19
4,data_report_22
5,data_report_35
6,data_report_41
7,data_report_42
8,data_report_43
9,data_report_45


In [12]:
import mysql.connector
import pandas as pd
conn = mysql.connector.connect(user='admin', database='db',
                               password='admin',
                               host="192.168.14.91",
                               port=3306)
# cursor = conn.cursor()
query = "SELECT count(*) from data_report;"
table_names = pd.read_sql(query, con=conn)
table_names.iloc[0]

count(*)    46588
Name: 0, dtype: int64

In [ ]:
query = "SELECT * FROM data_report_13 limit 10"
table_names = pd.read_sql(query, con=conn)
table_names

In [42]:
conn = mysql.connector.connect(user='tsb_user', database='tsb_gps_db',
                               password='tSb@2022!',
                               host="202.129.206.52",
                               port=3306)
cursor = conn.cursor()
query = "SELECT table_name FROM information_schema.tables WHERE table_type = 'base table'"
table_names = pd.read_sql(query, con=conn)
conn.close()

for i in [4]:
    if i==9:
        start = f"0{i}"
        end = f"{i+1}"
    else:
        start = f"0{i}"
        end = f"0{i+1}"

    query = f"""
    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[0][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[1][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[2][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[3][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[4][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[5][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[6][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[7][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[8][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[9][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[10][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"

    union all

    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[11][0]}
    where "2022-{start}-01" < ADDTIME(data_date, "7:00:00") and ADDTIME(data_date, "7:00:00") < "2022-{end}-01"
    """

In [43]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(user='tsb_user', database='tsb_gps_db',
                               password='tSb@2022!',
                               host="202.129.206.52",
                               port=3306)
cursor = conn.cursor()

# Create a pandas dataframe
df = pd.read_sql(query, con=conn)
df['gps_imei'] = df.gps_imei.astype(int)
# df.merge(all_v, how='left', on='gps_imei').dropna(axis=0).reset_index(drop=True)
conn.close()

print(df)

                        utc         gps_imei        lat         lon  speed
0       2022-04-01 09:07:54  863835027506307  13.745457  100.516452    0.0
1       2022-04-01 09:08:54  863835027506307  13.745933  100.514322   34.0
2       2022-04-01 09:06:54  863835027506307  13.744642  100.516318    0.0
3       2022-04-01 09:05:54  863835027506307  13.743140  100.516097    0.0
4       2022-04-01 09:03:54  863835027506307  13.742552  100.516035   37.0
...                     ...              ...        ...         ...    ...
8693966 2022-04-30 23:01:16  869530043173083  13.663177  100.514682    0.0
8693967 2022-04-30 23:11:16  869530043173083  13.663177  100.514682    0.0
8693968 2022-04-30 23:21:16  869530043173083  13.663177  100.514682    0.0
8693969 2022-04-30 23:31:16  869530043173083  13.663177  100.514682    0.0
8693970 2022-04-30 23:41:16  869530043173083  13.663177  100.514682    0.0

[8693971 rows x 5 columns]


In [44]:
df_m = df.merge(all_v, how='left', on='gps_imei').dropna(axis=0).reset_index(drop=True)
df_m.head()

,utc,gps_imei,lat,lon,speed,registerNumber,chassisNumber,motorNumber,brand,status,description,route
0,2022-04-01 09:07:54,863835027506307,13.745457,100.516452,0.0,16-5876 กทม.,MRSBCREMXMZM00013,MCS20111173,MINE BUS,active,รร.ศึกษานารีวิทยา 2 - หัวลำโพง,4-36(7)
1,2022-04-01 09:08:54,863835027506307,13.745933,100.514322,34.0,16-5876 กทม.,MRSBCREMXMZM00013,MCS20111173,MINE BUS,active,รร.ศึกษานารีวิทยา 2 - หัวลำโพง,4-36(7)
2,2022-04-01 09:06:54,863835027506307,13.744642,100.516318,0.0,16-5876 กทม.,MRSBCREMXMZM00013,MCS20111173,MINE BUS,active,รร.ศึกษานารีวิทยา 2 - หัวลำโพง,4-36(7)
3,2022-04-01 09:05:54,863835027506307,13.743140,100.516097,0.0,16-5876 กทม.,MRSBCREMXMZM00013,MCS20111173,MINE BUS,active,รร.ศึกษานารีวิทยา 2 - หัวลำโพง,4-36(7)
4,2022-04-01 09:03:54,863835027506307,13.742552,100.516035,37.0,16-5876 กทม.,MRSBCREMXMZM00013,MCS20111173,MINE BUS,active,รร.ศึกษานารีวิทยา 2 - หัวลำโพง,4-36(7)


In [45]:
df_m.route.unique()

array(['4-36(7)', '4-40(56)', '4-1(6)', '3-15(133)', '4-44(80ก.)',
       '4-21(120)', '1-5(39)', '4-8(35)'], dtype=object)

In [46]:
for i in df_m.route.unique():
    print(i,len(df_m[df_m['route'] == i]))

4-36(7) 1493281
4-40(56) 793220
4-1(6) 1333609
3-15(133) 1333987
4-44(80ก.) 981282
4-21(120) 362151
1-5(39) 946072
4-8(35) 1145617


# Check start & end time

In [ ]:
import mysql.connector
conn = mysql.connector.connect(user='tsb_user', database='tsb_gps_db',
                               password='tSb@2022!',
                               host="202.129.206.52",
                               port=3306)
# cursor = conn.cursor()
query = "SELECT table_name FROM information_schema.tables WHERE table_type = 'base table'"
table_names = pd.read_sql(query, con=conn)
table_names

In [17]:
from tqdm import tqdm
table = []
time_all = []
for i in tqdm(range(len(table_names))):
    query = f"""
    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[i][0]}
    order by utc DESC
    limit 5
    """

    df = pd.read_sql(query, con=conn)
    if len(df) > 0:
        if table_names.iloc[i][0] == 'data_report_19':
            time = df.iloc[1].utc
    #         print(table_names.toPandas().iloc[i][0], time)
            table.append(table_names.iloc[i][0])
            time_all.append(time)
        else:
            time = df.iloc[0].utc
    #         print(table_names.toPandas().iloc[i][0], time)
            table.append(table_names.iloc[i][0])
            time_all.append(time)

100%|██████████| 12/12 [00:29<00:00,  2.46s/it]


In [18]:
import pandas as pd
end = pd.DataFrame([table, time_all]).T
end.columns = ['data_report', 'end_time']
end

,data_report,end_time
0,data_report_13,2022-09-21 20:59:52
1,data_report_14,2022-09-21 20:52:20
2,data_report_16,2022-09-21 20:55:24
3,data_report_19,2022-09-21 21:09:14
4,data_report_22,2022-09-21 21:09:39
5,data_report_35,2022-09-21 21:19:43
6,data_report_41,2022-09-21 21:19:42
7,data_report_42,2022-09-21 21:18:56
8,data_report_45,2022-09-21 21:03:13
9,data_report_47,2022-09-21 21:49:31


In [19]:
from tqdm import tqdm
table = []
time_all = []
for i in tqdm(range(len(table_names))):
    query = f"""
    select
        ADDTIME(data_date, "7:00:00") as utc,
        box_imei as gps_imei,
        st_x(g_point) as lat, 
        st_y(g_point) as lon,
        speed
    FROM {table_names.iloc[i][0]}
    order by utc ASC
    limit 5
    """

    df = pd.read_sql(query, con=conn)
    if len(df) > 0:
        if table_names.iloc[i][0] == 'data_report_19':
            time = df.iloc[1].utc
    #         print(table_names.toPandas().iloc[i][0], time)
            table.append(table_names.iloc[i][0])
            time_all.append(time)
        else:
            time = df.iloc[0].utc
    #         print(table_names.toPandas().iloc[i][0], time)
            table.append(table_names.iloc[i][0])
            time_all.append(time)

100%|██████████| 12/12 [00:29<00:00,  2.45s/it]


In [22]:
import pandas as pd
start = pd.DataFrame([table, time_all]).T
start.columns = ['data_report', 'start_time']
start

,data_report,start_time
0,data_report_13,2022-03-25 00:00:23
1,data_report_14,2022-03-25 00:00:59
2,data_report_16,2022-05-07 15:52:45
3,data_report_19,2022-06-22 14:39:44
4,data_report_22,2022-03-24 00:00:01
5,data_report_35,2022-03-25 00:00:05
6,data_report_41,2022-03-25 00:00:57
7,data_report_42,2022-03-25 00:01:02
8,data_report_45,2022-03-24 00:11:06
9,data_report_47,2022-03-21 07:12:22


In [23]:
start_end_time = start.merge(end, left_on='data_report', right_on='data_report')
start_end_time

,data_report,start_time,end_time
0,data_report_13,2022-03-25 00:00:23,2022-09-21 20:59:52
1,data_report_14,2022-03-25 00:00:59,2022-09-21 20:52:20
2,data_report_16,2022-05-07 15:52:45,2022-09-21 20:55:24
3,data_report_19,2022-06-22 14:39:44,2022-09-21 21:09:14
4,data_report_22,2022-03-24 00:00:01,2022-09-21 21:09:39
5,data_report_35,2022-03-25 00:00:05,2022-09-21 21:19:43
6,data_report_41,2022-03-25 00:00:57,2022-09-21 21:19:42
7,data_report_42,2022-03-25 00:01:02,2022-09-21 21:18:56
8,data_report_45,2022-03-24 00:11:06,2022-09-21 21:03:13
9,data_report_47,2022-03-21 07:12:22,2022-09-21 21:49:31


In [39]:
df_m.sort_values('utc')

,utc,gps_imei,lat,lon,speed,num,routes,chasis
3544454,2022-09-01 00:00:01,866381052390660,13.754578,100.423293,0.0,16-5960 กทม.,56,MRSBCREM3MZM00094
1494294,2022-09-01 00:00:03,868998030615351,13.662763,100.514728,0.0,16-5844 กทม.,35,MRSBCREM1MZM00062
2952151,2022-09-01 00:00:10,868998030633305,13.754737,100.423368,0.0,16-5867 กทม.,56,MRSBCREM4MZM00055
2537053,2022-09-01 00:00:15,864507039676258,13.663248,100.514987,0.0,16-5854 กทม.,6,MRSBCREM0MZM00053
2537052,2022-09-01 00:00:15,864507039676258,13.663248,100.514987,0.0,16-5854 กทม.,6,MRSBCREM0MZM00053
...,...,...,...,...,...,...,...,...
4512191,2022-09-21 21:48:46,867553050797893,14.091068,100.618530,55.0,16-5969 กทม.,39,MRSBCREM8MZM00091
4756204,2022-09-21 21:49:00,869530043171129,13.998537,100.614642,46.0,16-5966 กทม.,39,MRSBCREM6MZM00087
4512192,2022-09-21 21:49:01,867553050797893,14.088937,100.618448,57.0,16-5969 กทม.,39,MRSBCREM8MZM00091
4512193,2022-09-21 21:49:16,867553050797893,14.086915,100.618382,47.0,16-5969 กทม.,39,MRSBCREM8MZM00091


In [18]:
df_m[df_m.routes==39]

,utc,gps_imei,lat,lon,speed,num,routes,chasis
1375322,2022-09-01 07:09:38,864507039651681,13.996012,100.581745,0.0,16-5883 กทม.,39,MRSBCREM6MZM00073
1375323,2022-09-01 07:09:23,864507039651681,13.996012,100.581745,0.0,16-5883 กทม.,39,MRSBCREM6MZM00073
1375324,2022-09-01 07:09:08,864507039651681,13.996012,100.581745,0.0,16-5883 กทม.,39,MRSBCREM6MZM00073
1375325,2022-09-01 07:08:53,864507039651681,13.996012,100.581745,0.0,16-5883 กทม.,39,MRSBCREM6MZM00073
1375326,2022-09-01 07:08:38,864507039651681,13.996012,100.581745,0.0,16-5883 กทม.,39,MRSBCREM6MZM00073
...,...,...,...,...,...,...,...,...
4756200,2022-09-21 21:48:00,869530043171129,13.991343,100.614847,45.0,16-5966 กทม.,39,MRSBCREM6MZM00087
4756201,2022-09-21 21:48:15,869530043171129,13.993113,100.614565,48.0,16-5966 กทม.,39,MRSBCREM6MZM00087
4756202,2022-09-21 21:48:30,869530043171129,13.994960,100.614527,48.0,16-5966 กทม.,39,MRSBCREM6MZM00087
4756203,2022-09-21 21:48:45,869530043171129,13.996767,100.614587,47.0,16-5966 กทม.,39,MRSBCREM6MZM00087


In [ ]:
# df.to_csv(f'/home/jovyan/work/data/gps_thaistar/gps_m{start}.csv',index=False)

In [154]:
df[df.speed!=0].gps_imei.unique()

array([863835027506307, 868998030618942, 867648045377536, 866381052358725,
       866381052358758, 866381052370019, 866381052370092, 866381052370134,
       866381052370167, 866381052370209, 866381052370225, 866381052370274,
       866381052370324, 866381052370340, 866381052370738, 866381052373542,
       866381052388714, 866381052388771, 866381052394605, 866381052394613,
       866381052394621, 866381052394969, 866381052395008, 866381052395891,
       866381052396626, 866381052396634, 866381052396675, 866381052396683,
       866381052396907, 866381052397095, 866381052397160, 866381052397178,
       868998030587931, 864507039674154, 864507039673792, 864507039665376,
       864507039655831, 864507039651681, 864507039648240, 863835027811913,
       868998031822683, 868998030646927, 868998030639609, 868998030634774,
       868998030615351, 868998030594143, 864507039673982, 864507039711451,
       864507039650204, 868998030639021, 868998030627521, 864507032382094,
       863835026096490, 8

In [144]:
gps_list = df.gps_imei.tolist()

In [106]:
# set.intersection(set(v_list),set(gps_list)), set.intersection(set(sb_list),set(gps_list))

In [145]:
# if intersect = False, if disjoint = True
set(v_list).isdisjoint(set(gps_list)), set(sb_list).isdisjoint(set(gps_list))

(False, True)

In [146]:
set(vehicle_gps_list).isdisjoint(set(gps_list))

False

In [159]:
all_v[all_v.routes == 56].gps_imei.tolist()

[864507039711535,
 868998030618942,
 868998030587691,
 868998030594861,
 868998030633305,
 868998030633206,
 868998030635771,
 867553050796408,
 866381052390660,
 869530043171988,
 866381052393128,
 867553050806694]